In [1]:
import pathlib
import os
if not "changed_working_directory" in locals():
    changed_working_directory = True
    os.chdir(pathlib.Path().resolve().parent)
    
%load_ext autoreload
%autoreload 2

In [2]:
from tabGAN import TabGAN, TabGANcf
from src import constants as const
import helpers

imports_path = "src//imports.ipynb"
tabGAN_path = "tabGAN.py"
utils_path = "utils\\utils.ipynb"

%run "$imports_path"

if const.dir.project() != os.getcwd():
    raise ValueError("Project directory not the same as in global vars")

In [109]:
dataset_train_path = os.path.join(const.dir.data(), "df_adult_edited_train.csv")
dataset_test_path = os.path.join(const.dir.data(), "df_adult_edited_test.csv")

data_train = pd.read_csv(dataset_train_path)
data_test = pd.read_csv(dataset_test_path)
discrete_columns = data_train.columns[data_train.dtypes == "object"]

In [130]:
data_train

,age,workclass,fnlwgt,education,educational.num,marital.status,occupation,race,gender,capital.gain,capital.loss,hours.per.week,native.country,income
0,39,Government,77516,Bachelors,13,Never-married,Adm-clerical,White,Male,2174,0,40,US,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married,Exec-managerial,White,Male,0,0,13,US,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,White,Male,0,0,40,US,<=50K
3,53,Private,234721,<=12th,7,Married,Handlers-cleaners,Black,Male,0,0,40,US,<=50K
4,28,Private,338409,Bachelors,13,Married,Prof-specialty,Black,Female,0,0,40,Non-US,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30157,27,Private,257302,Assoc-acdm,12,Married,Tech-support,White,Female,0,0,38,US,<=50K
30158,40,Private,154374,HS-grad,9,Married,Machine-op-inspct,White,Male,0,0,40,US,>50K
30159,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,White,Female,0,0,40,US,<=50K
30160,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,White,Male,0,0,20,US,<=50K


In [133]:
tgcf.columns_discrete

Index(['workclass', 'education', 'marital.status', 'occupation', 'race',
       'gender', 'native.country', 'income'],
      dtype='object')

In [139]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer, LabelEncoder
from sklearn.compose import make_column_transformer
transformers = make_column_transformer(
    (FunctionTransformer(lambda x: x, lambda x:x), ['age', 'fnlwgt', 'educational.num', 'capital.gain', 'capital.loss',
       'hours.per.week']),
    (OneHotEncoder(drop="if_binary", sparse=False), ['workclass', 'education', 'marital.status', 'occupation', 'race',
       'gender', 'native.country']))
X = transformers.fit_transform(data_train.drop("income", axis=1))
label_encoder = LabelEncoder()
Y = label_encoder.fit_transform(data_train["income"])
Y

array([0, 0, 0, ..., 0, 0, 1])

In [150]:
a = tgcf.sample_scaled()

In [153]:
tgcf.inv_data_transform(a)

TypeError: inv_data_transform() missing 1 required positional argument: 'data_discrete_oh'

In [141]:
reg = LinearRegression().fit(X, Y)
reg.score(X, Y)

0.35912885397524774

In [92]:
reg.predict([[0,0]])

array([3.])

In [51]:
import sklearn
?sklearn.linear_model.LinearRegression

In [127]:
tgcf = TabGANcf(data_train, lambda x: np.random.uniform(size=x.shape[0]), n_critic=10,
               quantile_transformation_int=True, quantile_rand_transformation=True, noise_discrete_unif_max=0.4)
tgcf.train(10, progress_bar=True)

  0%|          | 0/10 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [172]:
with tf.GradientTape() as tape:
    a = tf.constant(5.)
    print(a)
    b = tf.cast(tf.numpy_function(lambda: np.array(7.), [], tf.float32), dtype=tf.float32)
    print(b)
    c = a * b
print(tape.gradient(c, a))

tf.Tensor(5.0, shape=(), dtype=float32)
tf.Tensor(7.0, shape=(), dtype=float32)
None


In [5]:
tgcf.sample()

,age,workclass,fnlwgt,education,educational.num,marital.status,occupation,race,gender,capital.gain,capital.loss,hours.per.week,native.country,income
0,36.000000,Government,39203.953125,Assoc-voc,13.0,Never-married,Exec-managerial,White,Male,0.0,0.0,40.0,US,<=50K
1,35.000000,Private,208880.671875,<=12th,7.0,Never-married,Tech-support,White,Male,0.0,0.0,40.0,US,<=50K
2,28.000000,Private,43067.070312,<=12th,4.0,Married,Other-service,White,Male,0.0,0.0,40.0,US,<=50K
3,55.000000,Private,208543.250000,HS-grad,9.0,Widowed,Machine-op-inspct,White,Male,0.0,0.0,18.0,US,<=50K
4,28.000000,Private,155674.875000,Bachelors,10.0,Separated,Adm-clerical,White,Male,0.0,0.0,48.0,US,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30157,30.000000,Private,551353.562500,Some-college,10.0,Never-married,Handlers-cleaners,White,Female,0.0,0.0,60.0,Non-US,<=50K
30158,46.290089,Private,180144.843750,Bachelors,16.0,Married,Prof-specialty,Asian-Pac-Islander,Male,0.0,0.0,40.0,Non-US,>50K
30159,22.000000,Private,346421.875000,Some-college,9.0,Married,Craft-repair,White,Female,0.0,0.0,30.0,US,>50K
30160,33.000000,Government,119276.476562,Assoc-acdm,13.0,Married,Farming-fishing,White,Female,0.0,0.0,40.0,US,<=50K
